### Import libraries and create DataFrame from csv 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("WR_Data_2024.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,G,FPTS,FPTS/G
0,1,Chris Godwin (TB),43,53,511,11.9,55,6,5,1,2,0,6,124.3,20.7
1,2,Ja'Marr Chase (CIN),34,42,565,16.6,70,8,5,0,0,0,6,120.5,20.1
2,3,Nico Collins (HOU),32,45,567,17.7,67,8,3,0,0,0,5,106.7,21.3
3,4,Jayden Reed (GB),27,34,442,16.4,70,8,3,9,110,1,6,106.2,17.7
4,5,Drake London (ATL),38,54,428,11.3,31,2,4,0,0,0,6,104.8,17.5
5,6,Stefon Diggs (HOU),37,48,392,10.6,28,4,3,2,12,1,6,101.9,17.0
6,7,Garrett Wilson (NYJ),41,67,399,9.7,26,4,3,1,5,0,6,97.4,16.2
7,8,Justin Jefferson (MIN),26,43,450,17.3,97,8,4,0,0,0,5,95.0,19.0
8,9,CeeDee Lamb (DAL),32,55,467,14.6,65,6,2,8,37,0,6,92.4,15.4
9,10,Malik Nabers (NYG),35,52,386,11.0,39,7,3,3,0,0,4,91.6,22.9


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [2]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/R', 'LG', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,Y/R,LG,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game,ATT/game,YDS.1/game,TD.1/game
0,1,Chris Godwin (TB),11.9,55,6,124.3,20.7,7.2,8.8,85.2,1.0,0.8,0.2,0.3,0.0
1,2,Ja'Marr Chase (CIN),16.6,70,6,120.5,20.1,5.7,7.0,94.2,1.3,0.8,0.0,0.0,0.0
2,3,Nico Collins (HOU),17.7,67,5,106.7,21.3,6.4,9.0,113.4,1.6,0.6,0.0,0.0,0.0
3,4,Jayden Reed (GB),16.4,70,6,106.2,17.7,4.5,5.7,73.7,1.3,0.5,1.5,18.3,0.2
4,5,Drake London (ATL),11.3,31,6,104.8,17.5,6.3,9.0,71.3,0.3,0.7,0.0,0.0,0.0
5,6,Stefon Diggs (HOU),10.6,28,6,101.9,17.0,6.2,8.0,65.3,0.7,0.5,0.3,2.0,0.2
6,7,Garrett Wilson (NYJ),9.7,26,6,97.4,16.2,6.8,11.2,66.5,0.7,0.5,0.2,0.8,0.0
7,8,Justin Jefferson (MIN),17.3,97,5,95.0,19.0,5.2,8.6,90.0,1.6,0.8,0.0,0.0,0.0
8,9,CeeDee Lamb (DAL),14.6,65,6,92.4,15.4,5.3,9.2,77.8,1.0,0.3,1.3,6.2,0.0
9,10,Malik Nabers (NYG),11.0,39,4,91.6,22.9,8.8,13.0,96.5,1.8,0.8,0.8,0.0,0.0


### Calculate the correlations for the final stats across different conditions

In [3]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/R,0.62,0.22,0.05,0.14,0.26
LG,0.82,0.66,0.23,0.23,0.49
REC/game,0.95,0.94,0.76,0.64,0.82
TGT/game,0.94,0.93,0.60,0.40,0.72
YDS/game,0.98,0.97,0.87,0.85,0.92
20+/game,0.85,0.82,0.54,0.57,0.70
TD/game,0.84,0.83,0.74,0.63,0.76
ATT/game,0.27,0.23,0.13,0.02,0.16
YDS.1/game,0.24,0.20,0.11,-0.06,0.12
TD.1/game,0.19,0.16,0.06,0.15,0.14


### Assign the weights for the final stats

In [4]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/R,0.26,0.07,0.07
LG,0.49,0.24,0.08
REC/game,0.82,0.68,0.15
TGT/game,0.72,0.51,0.13
YDS/game,0.92,0.84,0.17
20+/game,0.70,0.49,0.09
TD/game,0.76,0.58,0.13
ATT/game,0.16,0.03,0.06
YDS.1/game,0.12,0.02,0.06
TD.1/game,0.14,0.02,0.06


### Multiply the assigned weights to the final stats and calculate the score

In [5]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]
df_weight.sort_values(by=['Rank'], ascending=True).head(10)

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,LG_weighted,REC/game_weighted,TGT/game_weighted,YDS/game_weighted,20+/game_weighted,TD/game_weighted
0,1,Chris Godwin (TB),20.7,4.2,1.1,1.1,14.2,0.1,0.1
1,2,Ja'Marr Chase (CIN),20.1,5.4,0.8,0.9,15.7,0.1,0.1
2,3,Nico Collins (HOU),21.3,5.2,0.9,1.1,18.9,0.1,0.1
3,4,Jayden Reed (GB),17.7,5.4,0.7,0.7,12.3,0.1,0.1
4,5,Drake London (ATL),17.5,2.4,0.9,1.1,11.9,0.0,0.1
5,6,Stefon Diggs (HOU),17.0,2.2,0.9,1.0,10.9,0.1,0.1
6,7,Garrett Wilson (NYJ),16.2,2.0,1.0,1.4,11.1,0.1,0.1
7,8,Justin Jefferson (MIN),19.0,7.5,0.8,1.1,15.0,0.1,0.1
8,9,CeeDee Lamb (DAL),15.4,5.0,0.8,1.2,13.0,0.1,0.0
9,10,Malik Nabers (NYG),22.9,3.0,1.3,1.6,16.1,0.2,0.1


### Model training

In [6]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 0.5153236342883896
Test Set MSE: 0.583507467110238
Random Forest Test Set MSE: 0.10108926424035554


/tmp/ipykernel_1748/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [7]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("WR_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game
Rank,,,,,,,,,,,,
3,1,Nico Collins (HOU),10.00,2,5,106.7,21.3,6.4,9.0,113.4,1.6,0.6
8,2,Justin Jefferson (MIN),9.46,6,5,95.0,19.0,5.2,8.6,90.0,1.6,0.8
2,3,Ja'Marr Chase (CIN),9.21,-1,6,120.5,20.1,5.7,7.0,94.2,1.3,0.8
1,4,Chris Godwin (TB),9.08,-3,6,124.3,20.7,7.2,8.8,85.2,1.0,0.8
61,5,A.J. Brown (PHI),8.92,56,2,46.5,23.3,5.5,9.5,117.5,2.0,1.0
10,6,Malik Nabers (NYG),8.54,4,4,91.6,22.9,8.8,13.0,96.5,1.8,0.8
67,7,Cooper Kupp (LAR),8.36,60,2,39.7,19.9,9.0,13.5,73.5,1.5,0.5
5,8,Drake London (ATL),7.57,-3,6,104.8,17.5,6.3,9.0,71.3,0.3,0.7
29,9,DeVonta Smith (PHI),7.57,20,4,66.3,16.6,6.0,8.0,75.8,0.8,0.5
